In [2]:
# from sqlalchemy import create_engine
import pandas as pd
import numpy as np
# from binance import Client
import matplotlib.pyplot as plt
# import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
import sys
import ta
import yaml
from data import get_data
import os
sys.path.insert(1, '/Users/arsenchik/Desktop/dipploma/machine_learning_in_hft/algotrading/backtesting')
from backtest import * 
from strategies import *
from config import *
# color_pal = sns.color_palette()
# plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import (
    log_loss,
    roc_auc_score,
    f1_score,
    mean_squared_error,
    accuracy_score,
    r2_score,
    mean_absolute_error,
)

In [2]:
FEATURES = ['Open',
    'High',
    'Low',
    'Volume',
    'RSI_2',
    'SMA_20',
    'RSI_4',
    'SMA_40',
    'RSI_6',
    'SMA_60',
    'RSI_8',
    'SMA_80']
TARGET = 'Target'

In [3]:
df = pd.read_csv('data.csv')
df.set_index("Time", inplace=True)

In [10]:
df.head(3)

,Open,High,Low,Close,Volume,RSI_2,SMA_20,RSI_4,SMA_40,RSI_6,SMA_60,RSI_8,SMA_80,Target
Time,,,,,,,,,,,,,,
2023-01-09 05:56:00,17215.84,17216.97,17209.06,17209.27,115.13422,14.826789,17215.3595,33.825777,17226.18875,37.700881,17227.123333,38.189317,17228.381250,0
2023-01-09 05:57:00,17209.60,17211.78,17205.53,17206.49,208.19562,9.218733,17214.1490,26.927548,17225.27575,32.247758,17226.787000,33.907212,17228.096125,1
2023-01-09 05:58:00,17207.01,17208.40,17202.76,17207.36,122.58709,26.596256,17213.3830,32.658002,17224.57750,35.738367,17226.486000,36.455564,17227.850750,0


In [13]:
for i in range(5):
    df[f'close_{i+1}'] = df['Close'].shift(i+1)
df.head(7)

,Open,High,Low,Close,Volume,RSI_2,SMA_20,RSI_4,SMA_40,RSI_6,SMA_60,RSI_8,SMA_80,Target,close_1,close_2,close_3,close_4,close_5
Time,,,,,,,,,,,,,,,,,,,
2023-01-09 05:56:00,17215.84,17216.97,17209.06,17209.27,115.13422,14.826789,17215.3595,33.825777,17226.18875,37.700881,17227.123333,38.189317,17228.381250,0,NaN,NaN,NaN,NaN,NaN
2023-01-09 05:57:00,17209.60,17211.78,17205.53,17206.49,208.19562,9.218733,17214.1490,26.927548,17225.27575,32.247758,17226.787000,33.907212,17228.096125,1,17209.27,NaN,NaN,NaN,NaN
2023-01-09 05:58:00,17207.01,17208.40,17202.76,17207.36,122.58709,26.596256,17213.3830,32.658002,17224.57750,35.738367,17226.486000,36.455564,17227.850750,0,17206.49,17209.27,NaN,NaN,NaN
2023-01-09 05:59:00,17207.37,17207.37,17202.92,17205.95,152.32090,16.412670,17212.4085,27.925648,17223.73975,32.483580,17226.143667,34.025584,17227.574500,1,17207.36,17206.49,17209.27,NaN,NaN
2023-01-09 06:00:00,17205.95,17211.70,17203.99,17207.22,188.37680,50.532869,17211.6610,38.609170,17222.67725,38.534040,17225.826500,38.261721,17227.318500,1,17205.95,17207.36,17206.49,17209.27,NaN
2023-01-09 06:01:00,17207.22,17214.10,17205.84,17207.29,380.94533,52.662952,17210.9650,39.270724,17221.88925,38.896218,17225.491167,38.510424,17226.953750,0,17207.22,17205.95,17207.36,17206.49,17209.27
2023-01-09 06:02:00,17207.29,17220.59,17204.28,17206.08,349.19005,21.161119,17210.1965,31.457768,17221.01700,34.659941,17225.178667,35.671654,17226.573875,1,17207.29,17207.22,17205.95,17207.36,17206.49


In [5]:
from dataset import *

data = Data(df.drop(['RSI_2','RSI_4','RSI_6','RSI_8', 'SMA_20', 'SMA_40', 'SMA_60', 'SMA_80'], axis=1))

data.indicators_calc()
data.create_features()
data.df

,Open,High,Low,Close,Volume,Target,RSI_2,SMA_20,RSI_4,SMA_40,RSI_6,SMA_60,RSI_8,SMA_80,MACD,close_1,close_2,close_3,close_4,close_5
Time,,,,,,,,,,,,,,,,,,,,
2023-01-09 07:15:00,17192.59,17194.80,17190.94,17191.91,71.38414,0,23.272898,17190.9800,47.994138,17194.83175,51.723542,17199.035333,51.476169,17200.585500,0.908350,17192.58,17194.48,17194.30,17192.33,17190.82
2023-01-09 07:16:00,17191.91,17193.00,17190.88,17191.28,45.02310,0,14.642676,17190.9380,41.364733,17194.37350,47.595062,17198.754500,48.678064,17200.360625,0.760926,17191.91,17192.58,17194.48,17194.30,17192.33
2023-01-09 07:17:00,17191.38,17191.76,17189.65,17190.97,60.30235,1,10.727699,17190.8810,37.927562,17193.92825,45.452836,17198.514667,47.234196,17200.166625,0.623910,17191.28,17191.91,17192.58,17194.48,17194.30
2023-01-09 07:18:00,17190.97,17192.96,17190.03,17192.51,83.82605,0,75.584822,17190.9465,59.963280,17193.52475,56.992389,17198.267500,54.839309,17199.981000,0.617239,17190.97,17191.28,17191.91,17192.58,17194.48
2023-01-09 07:19:00,17192.51,17192.90,17191.00,17192.14,55.28644,1,56.026038,17191.0915,53.840391,17192.98525,53.716091,17198.023500,52.751638,17199.808375,0.569318,17192.51,17190.97,17191.28,17191.91,17192.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-19 20:32:00,24352.13,24364.22,24337.53,24359.96,214.97637,1,70.039155,24382.1570,49.674985,24428.80650,45.690400,24465.877833,44.246869,24482.549875,0.248195,24352.13,24354.57,24321.94,24341.52,24377.74
2023-02-19 20:33:00,24360.76,24411.43,24350.81,24406.65,275.03299,1,94.063230,24380.4885,74.412209,24425.49425,64.396187,24463.462167,58.912317,24480.821875,4.122665,24359.96,24352.13,24354.57,24321.94,24341.52
2023-02-19 20:34:00,24408.41,24431.85,24405.58,24418.98,291.29097,1,95.829479,24380.1065,78.187500,24422.49900,67.899883,24461.313333,61.934294,24479.262125,7.353218,24406.65,24359.96,24352.13,24354.57,24321.94


In [20]:
df.shape

(59921, 15)

In [ ]:
import pandas as pd
from model import *
import warnings
from dataset import *
warnings.filterwarnings("ignore")
df = pd.read_csv('data.csv')
df.set_index("Time", inplace=True)
data = Data(df)
data.indicators_calc()
data.create_target(task_type='classification')
model = Model(model_name="LightGBM", task_type="classification")
# output_dir_path = '/Users/arsenchik/Desktop/dipploma/machine_learning_in_hft/algotrading/model_training/optimization_results'
output_dir_path = '/home/kenny/algotrading/model_training/optimization_results'
model.optimize(data, output_dir_path, number_of_trials=50)

In [17]:
tss = TimeSeriesSplit(n_splits=5, test_size=3*24*60)
df = df.sort_index()

fold = 0
preds = []
scores = []

for train_idx, val_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[val_idx]

    X_train = train[FEATURES]
    y_train = train[TARGET]

    X_test = test[FEATURES]
    y_test = test[TARGET]

    clf = lgb.LGBMClassifier(n_estimators=1000)
    clf.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            verbose=100)

    y_pred = clf.predict(X_test)
    preds.append(y_pred)
    score = f1_score(y_test, y_pred)
    scores.append(score)

[100]	training's binary_logloss: 0.652832	valid_1's binary_logloss: 0.695833
[200]	training's binary_logloss: 0.62319	valid_1's binary_logloss: 0.698619
[300]	training's binary_logloss: 0.597564	valid_1's binary_logloss: 0.702441
[400]	training's binary_logloss: 0.574426	valid_1's binary_logloss: 0.706435
[500]	training's binary_logloss: 0.553463	valid_1's binary_logloss: 0.711089
[600]	training's binary_logloss: 0.532816	valid_1's binary_logloss: 0.714488
[700]	training's binary_logloss: 0.514628	valid_1's binary_logloss: 0.717471
[800]	training's binary_logloss: 0.496315	valid_1's binary_logloss: 0.719973
[900]	training's binary_logloss: 0.480044	valid_1's binary_logloss: 0.722234
[1000]	training's binary_logloss: 0.465002	valid_1's binary_logloss: 0.725299
[100]	training's binary_logloss: 0.655826	valid_1's binary_logloss: 0.697673
[200]	training's binary_logloss: 0.628066	valid_1's binary_logloss: 0.702078
[300]	training's binary_logloss: 0.603626	valid_1's binary_logloss: 0.704248

In [18]:
print(f'score across folds {np.mean(scores):0.4f}')
print(f'Fold scores:{scores}')

RMSE score across folds 0.4996
Fold scores:[0.5188419117647058, 0.5426714195848187, 0.48532948532948533, 0.4695270599707459, 0.4814723926380368]


In [ ]:
reg = xgb.XGBRegressor(booster='gbtree',    
                           n_estimators=1000,
                           early_stopping_rounds=50,
                           objective='reg:linear',
                           max_depth=3,
                           learning_rate=0.01)
reg.fit(data[FEATURES], data[TARGET],
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=100)